In [7]:
import os
import pandas as pd
import pickle as pkl
import time

import utils, get_nwp

In [8]:
config = utils.load_config('config.yaml')
db_config = config['write']['db_conf']

#recent_dfs = pkl.load(open(os.path.join('data', config['write']['recent_pkl']), 'rb'))
#master_pkl = pkl.load(open(os.path.join('data', config['write']['master_pkl']), 'rb'))

In [3]:
master_data = get_nwp.get_master_data(db_config=db_config)
locs = master_data[['longitude', 'latitude']]
locations = []
for lon, lat in locs.values:
    latitude, longitude = get_nwp.get_nearest_point(db_config=db_config,
                                            table='AnalysisFields',
                                            latitude=lat,
                                            longitude=lon)
    locations.append((latitude, longitude))

In [ ]:
lat, lon = master_data[master_data['station_id'] == '00164'][['latitude', 'longitude']].values[0]
nearest_lat, nearest_lon = get_nwp.get_nearest_point(db_config=db_config,
                                    table="singlelevelfields",
                                    latitude=lat,
                                    longitude=lon)
forecasts = get_nwp.query_for_location(config=config,
                                    table="multilevelfields",
                                    latitude=nearest_lat,
                                    longitude=nearest_lon)

In [ ]:
forecasts

In [ ]:
lat, lon = master_data[master_data['station_id'] == '00164'][['latitude', 'longitude']].values[0]
nearest_lat, nearest_lon = get_nwp.get_nearest_point(db_config=db_config,
                                    table="singlelevelfields",
                                    latitude=lat,
                                    longitude=lon)
w_vert = get_nwp.query_for_location(config=config,
                                    table="analysisfields",
                                    latitude=nearest_lat,
                                    longitude=nearest_lon)
w_vert['starttime'] = pd.to_datetime(w_vert['starttime'], utc=True)
w_vert['timestamp'] = w_vert['starttime'] + pd.to_timedelta(w_vert['forecasttime'], unit='h')
w_vert.set_index('timestamp', inplace=True)
w_vert.drop(['starttime', 'forecasttime', 'toplevel', 'bottomlevel'], axis=1, inplace=True)

In [ ]:
w_vert

In [6]:
# save locatiobns as pickle
with open('locations.pkl', 'wb') as f:
    pkl.dump(locations, f)

In [9]:
# now read the pickle file
with open('locations.pkl', 'rb') as f:
    locations = pkl.load(f)